# NeuraLogic Hooks

In [1]:
import numpy as np

from neuralogic.core import Template, Backend, Atom, Var
from neuralogic.core.constructs.predicate import Predicate
from neuralogic.utils.data import Data, Dataset
from neuralogic.core.settings import Settings, ErrorFunction, Optimizer
from neuralogic.nn import get_evaluator

from IPython.display import clear_output
import matplotlib.pyplot as plt

## Data preparation

In [2]:
src = np.array([
    1, 2, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 10, 10, 10, 11,
    12, 12, 13, 13, 13, 13, 16, 16, 17, 17, 19, 19, 21, 21, 25, 25, 27, 27,
    27, 28, 29, 29, 30, 30, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32,
    32, 32, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33
])

dst = np.array([
    0, 0, 1, 0, 1, 2, 0, 0, 0, 4, 5, 0, 1, 2, 3, 0, 2, 2, 0, 4, 5,
    0, 0, 3, 0, 1, 2, 3, 5, 6, 0, 1, 0, 1, 0, 1, 23, 24, 2, 23, 24, 2,
    23, 26, 1, 8, 0, 24, 25, 28, 2, 8, 14, 15, 18, 20, 22, 23, 29, 30,
    31, 8, 9, 13, 14, 15, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30, 31, 32
])

u = np.concatenate([src, dst])
v = np.concatenate([dst, src])
indices = [[i] for i in range(34)]

train_dataset = Dataset(data=[Data(x=np.ones((34,)), edge_index=[u, v], y=[[-1], [1]], y_mask=[[0], [33]])])
test_dataset = Dataset(data=[Data(x=np.ones((34,)), edge_index=[u, v], y=indices, y_mask=indices)])

In [3]:
settings = Settings(epochs=2, error_function=ErrorFunction.SQUARED_DIFF, optimizer=Optimizer.SGD)

## Model preparation

In [4]:
template = Template(settings=settings)

with template.context():
    template.add_rules([(Atom.node_feature_embed(i)[10, 1] <= Atom.node_feature(i)) for i in range(32)])

    template.add_rule(Atom.gcn_1(Var.X)[10, 10] <= (Atom.node_feature_embed(Var.Y), Atom.edge(Var.X, Var.Y)))
    template.add_rule(Atom.gcn_2(Var.X)[10, 10] <= (Atom.gcn_1(Var.Y), Atom.edge(Var.X, Var.Y)))
    template.add_rule(Atom.predict(Var.X)[np.ones((1, 10))] <= (Atom.gcn_2(Var.X)))


In [5]:
evaluator = get_evaluator(Backend.DYNET, template, settings)

There are no hooks set up - training/testing will work with no side effects

In [6]:
for _ in evaluator.train(train_dataset):
    pass

## Adding hooks

### Hooks declaration

Hooks are normal python functions with two parameters - `name` and `value`. `name` is the triggering name - which neuron triggered the hook and the `value` corresponds to the output of the neuron that triggered the hook.

In [7]:
def my_gcn2_hook(name, value):
    print("GCN2 hook triggered by", name)
    print("GCN2 hook value", "\n", value)


def my_gcn2_another_hook(name, value):
    print("MY ANOTHER GCN2 HOOK", "\n", value)


def my_predict_hook(name, value):
    print("Predict hook triggered by", name)
    print("Predict hook value", "\n", value)

### Attaching hooks

Hooks can be attached and dettached to a predicate in a template in multiple ways. All following ways are valid and will have the same result.

In [8]:
template.add_hook("gcn_2/1", my_gcn2_hook)

In [9]:
with template.context():
    template.add_hook(Predicate("gcn_2", 1), my_gcn2_hook)

In [10]:
with template.context():
    template.add_hook(Atom.gcn_2 / 1, my_gcn2_hook)

Dettaching hooks on a predicate can be done similary as attaching.

In [11]:
template.remove_hook("gcn_2/1", my_gcn2_hook)

In [12]:
with template.context():
    template.remove_hook(Predicate("gcn_2", 1), my_gcn2_hook)

In [13]:
with template.context():
    template.remove_hook(Atom.gcn_2 / 1, my_gcn2_hook)

### Example

Attach our hooks:

In [14]:
with template.context():
    template.add_hook(Atom.gcn_2 / 1, my_gcn2_hook)
    template.add_hook(Atom.gcn_2 / 1, my_gcn2_another_hook)  # We can add multiple hooks to one predicate
    template.add_hook(Atom.predict / 1, my_predict_hook)

Forward propagation will now trigger hooks when the value for the hooked predicate is being calculated

In [15]:
for _ in evaluator.train(train_dataset):
    print("\nEpoch trained\n")

MY ANOTHER GCN2 HOOK 
 [[ 0.92542136]
 [-0.23213652]
 [-0.80845255]
 [-0.94556266]
 [-0.75960463]
 [ 0.32345179]
 [-0.89326364]
 [ 0.36810002]
 [ 0.73240924]
 [-0.50499958]]
GCN2 hook triggered by WeightedAtomNeuron = gcn_2(0)
GCN2 hook value 
 [[ 0.92542136]
 [-0.23213652]
 [-0.80845255]
 [-0.94556266]
 [-0.75960463]
 [ 0.32345179]
 [-0.89326364]
 [ 0.36810002]
 [ 0.73240924]
 [-0.50499958]]
Predict hook triggered by WeightedAtomNeuron = predict(0)
Predict hook value 
 [[0.11435898]]
MY ANOTHER GCN2 HOOK 
 [[ 0.92490083]
 [-0.19250695]
 [-0.73887163]
 [-0.93078119]
 [-0.76095665]
 [ 0.31672907]
 [-0.87448519]
 [ 0.31192189]
 [ 0.72833532]
 [-0.46697885]]
GCN2 hook triggered by WeightedAtomNeuron = gcn_2(33)
GCN2 hook value 
 [[ 0.92490083]
 [-0.19250695]
 [-0.73887163]
 [-0.93078119]
 [-0.76095665]
 [ 0.31672907]
 [-0.87448519]
 [ 0.31192189]
 [ 0.72833532]
 [-0.46697885]]
Predict hook triggered by WeightedAtomNeuron = predict(33)
Predict hook value 
 [[0.11100036]]

Epoch trained

MY